## Importing the libraries




In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Loading the dataset

In [2]:
dataset = pd.read_csv("./Data.csv")
dataset.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


In [3]:
X = dataset.iloc[:, :-1].values # Independent variables, matrix of features.
Y = dataset.iloc[:, -1].values # Dependent variable(s), to be predicted.
print("X", X, "\n")
print("Y", Y, "\n")

X [['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]] 

Y ['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes'] 



## Handling missing data

In [4]:
from sklearn.impute import SimpleImputer
# We can choose one of many strategies, such as considering mean value, dropping the observation etc.
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer.fit(X[:, 1:3]) # Fit columns 'Age' and 'Salary', not 'Country' (considering columns of numerical values only).
X[:, 1:3] = imputer.transform(X[:, 1:3])
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

### Encoding the independent variable

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# We use one-hot encoder as there is no importance of order in the 'Country' column.
column_transformer = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough') # [0]: indices of columns to encode.
X = np.array(column_transformer.fit_transform(X))
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the dependent variable

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

In [7]:
print(Y)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into Training set and Test set

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [9]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [10]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [11]:
print(Y_train)

[0 1 0 0 1 1 0 1]


In [12]:
print(Y_test)

[0 1]


## Feature scaling

- Feature scaling should usually be applied after splitting the dataset into training set and test set.
- This is because the test set can be considered as a new set, so applying feature scaling before splitting may cause some information to leak from the test set into the training set.

In [13]:
from sklearn.preprocessing import StandardScaler
# We will use standardisation, which is more generic. Normalisation is used when data is normally distributed.
# We do not need to apply standardisation to dummy variables (which we encoded) as these values are already in a [-v, + v] (v ~ 3) range and
# their actual values (the encoding) has some meaning which would be lost on standardisation.
standard_scaler = StandardScaler()
# fit method computes the mean and standard deviation of each features, the transform method scales them.
X_train[:, 3:] = standard_scaler.fit_transform(X_train[:, 3:])

# We use the 'same scaler' which was applied to the training set, or any future set for that matter.
X_test[:, 3:] = standard_scaler.transform(X_test[:, 3:])

In [14]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [15]:
print(X_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
